#### This code serves to test Algorithm 5 for the Iterative Domain Information Framework

In [1]:
import glob as glob
import math, random
import pandas as pd 
from DatasetNames import get_dataset_class_name
from ExtraFunctions import *

from MultiClassDescription import MultiClassDescription

from MultiClassPredictionHybrid import MultiClassPredictionHybrid
from SingleClassPredictionHybrid import SingleClassPredictionHybrid

In [2]:
DATASET_NAME = "OrganCMNIST"
NUMBER_OF_IMAGES = 100 # Total number of images to be used
NUMBER_OF_AUGMENTATIONS = 2 # Number of augmentations to be used
number_of_experiment = 1 # Number of experiment to be conducted

source_directory = f"../Datasets/{DATASET_NAME}/test"  # Change this to your test directory path
_, image_paths = get_balanced_random_images(source_directory, n = NUMBER_OF_IMAGES)

random.seed(55)
random.shuffle(image_paths)
initial_image_paths = image_paths.copy()

dataset_classes_of_interest = get_dataset_class_name(DATASET_NAME)
init_classes_of_interest = list(dataset_classes_of_interest.values())

classes_of_interest_list = [random.sample(init_classes_of_interest, len(init_classes_of_interest)) for i in range(len(image_paths))]

multiclass_image_tasks = create_multiclass_augmented_image_tasks(image_paths, classes_of_interest_list, number_of_classes_to_predict=len(init_classes_of_interest), total_number_of_augmentations=NUMBER_OF_AUGMENTATIONS)

#### Playground

In [3]:
model_list = [
    #     {
    #     "model_name": "claude-3-5-sonnet-20241022",
    #     "model_type": "claude",
    # },
    # {
    #     "model_name": "gpt-4o",
    #     "model_type": "openai",
    # },
    {
        "model_name": "gpt-4o-mini",
        "model_type": "openai",
    }
]

In [4]:
mc_descriptor = MultiClassDescription(batch_size = 1, max_workers=2)
mcp_hybrid = MultiClassPredictionHybrid(batch_size = 1, max_workers=2)
single_hybrid = SingleClassPredictionHybrid(batch_size = 1, max_workers=2)

In [5]:
exp_results = []

for experiment in range(number_of_experiment):
    print(f"Experiment {experiment+1}/{number_of_experiment}")

    # Iterate over the models
    for model in model_list:
        model_name = model["model_name"]
        model_type = model["model_type"]

        print(f"Model: {model_name}")
        print(f"Model Type: {model_type}")

        multiclass_image_tasks = create_multiclass_augmented_image_tasks(image_paths, classes_of_interest_list, number_of_classes_to_predict=len(init_classes_of_interest), total_number_of_augmentations=NUMBER_OF_AUGMENTATIONS)

        mc_descriptor.update_model(model_name)
        mcp_hybrid.update_model(model_name)
        single_hybrid.update_model(model_name)

        # For each model, initialize the variables for the Algorithm
        n = len(init_classes_of_interest)
        k = 2
        threshold = 2
        number_of_classes_to_predict = math.ceil(n / k)
        image_paths = initial_image_paths
        voting_result_list = None    

        # Get the descriptions of the images
        rs = mc_descriptor.process_images(multiclass_image_tasks, model_type=model_type)   

        # Create Multiclass Image Tasks from the descriptions
        multiclass_image_tasks = []
        for item in rs['images']:
            image_path = item['image_path']
            classes_to_use = random.sample(init_classes_of_interest, len(init_classes_of_interest))

            # Combine all the descriptions into one so that it can be seen as a single medical report and also add headings to each section
            combined_description = f"Technical Quality: {item['technical_quality']}\nAnatomical Description: {item['anatomical_description']}\nFindings: {item['findings']}"
            
            task = MultiClassImageTask(image_path=image_path, classes=classes_to_use)
            task.set_image_textual_description(combined_description)
            n = len(init_classes_of_interest)
            k = 2
            task.set_num_predictions(number_of_classes_to_predict)
            task.encoded_image = item['encoded_image']
            multiclass_image_tasks.append(task)

        while number_of_classes_to_predict >= threshold:

            # Prompt the model to predict "number_of_classes_to_predict" classes for each image
            rs = mcp_hybrid.process_hybrid(multiclass_image_tasks, model_type=model_type)

            # Set the predicted classes as the new classes of interest
            result_list, image_paths, classes_of_interest_list = extract_image_classes_from_multiclass_predictions(rs, number_of_classes_to_predict)

            print("Image Paths: ", image_paths)
            print("Image Paths: ", len(image_paths))

            log_experiment_results(dataset_classes_of_interest, exp_results, experiment, model_name, model_type, number_of_classes_to_predict, image_paths, classes_of_interest_list)

            # Update the number of classes to predict
            number_of_classes_to_predict = math.ceil(number_of_classes_to_predict / k)

            multiclass_image_tasks = create_new_multitask_image_tasks(number_of_classes_to_predict, result_list)

        if rs:
            # Result from Multiclass Prediction
            result_list, _, _ = extract_image_classes_from_multiclass_predictions(rs)

            # Merge augmentation results
            result_list, voting_result_list = transform_result_list_to_single_image_tasks(result_list)
        else:
            # Go to SingleTask result list directly
            result_list = create_singleclass_image_tasks(image_paths, classes_of_interest_list)

        # Predict the single classes for each image
        f_res = single_hybrid.process_hybrid(result_list, model_type)
        correct_classes, pred_classes, image_paths = extract_single_classes(f_res, dataset_classes_of_interest)

        log_experiment_results(dataset_classes_of_interest, exp_results, experiment, model_name, model_type, number_of_classes_to_predict, image_paths, pred_classes, voting_result_list)

Experiment 1/1
Model: gpt-4o-mini
Model Type: openai
Image Paths:  ['../Datasets/OrganCMNIST/test\\2\\image_633.png', '../Datasets/OrganCMNIST/test\\7\\image_221.png', '../Datasets/OrganCMNIST/test\\4\\image_3788.png', '../Datasets/OrganCMNIST/test\\7\\image_4758.png', '../Datasets/OrganCMNIST/test\\9\\image_2534.png', '../Datasets/OrganCMNIST/test\\2\\image_4980.png', '../Datasets/OrganCMNIST/test\\6\\image_264.png', '../Datasets/OrganCMNIST/test\\4\\image_5065.png', '../Datasets/OrganCMNIST/test\\7\\image_5802.png', '../Datasets/OrganCMNIST/test\\8\\image_5936.png', '../Datasets/OrganCMNIST/test\\8\\image_3530.png', '../Datasets/OrganCMNIST/test\\5\\image_564.png', '../Datasets/OrganCMNIST/test\\7\\image_7186.png', '../Datasets/OrganCMNIST/test\\1\\image_1315.png', '../Datasets/OrganCMNIST/test\\6\\image_6127.png', '../Datasets/OrganCMNIST/test\\9\\image_7420.png', '../Datasets/OrganCMNIST/test\\0\\image_6892.png', '../Datasets/OrganCMNIST/test\\0\\image_1294.png', '../Datasets/Organ

ERROR:root:Max retries (3) exceeded: Connection error.


Batch processing failed: Connection error.
Failed batch: ['../Datasets/OrganCMNIST/test\\0\\image_6892.png']


ERROR:root:Max retries (3) exceeded: Connection error.


Batch processing failed: Connection error.
Failed batch: ['../Datasets/OrganCMNIST/test\\0\\image_1294.png']


Image Paths:  ['../Datasets/OrganCMNIST/test\\2\\image_633.png', '../Datasets/OrganCMNIST/test\\7\\image_221.png', '../Datasets/OrganCMNIST/test\\7\\image_4758.png', '../Datasets/OrganCMNIST/test\\4\\image_3788.png', '../Datasets/OrganCMNIST/test\\9\\image_2534.png', '../Datasets/OrganCMNIST/test\\2\\image_4980.png', '../Datasets/OrganCMNIST/test\\4\\image_5065.png', '../Datasets/OrganCMNIST/test\\6\\image_264.png', '../Datasets/OrganCMNIST/test\\8\\image_5936.png', '../Datasets/OrganCMNIST/test\\7\\image_5802.png', '../Datasets/OrganCMNIST/test\\8\\image_3530.png', '../Datasets/OrganCMNIST/test\\5\\image_564.png', '../Datasets/OrganCMNIST/test\\7\\image_7186.png', '../Datasets/OrganCMNIST/test\\1\\image_1315.png', '../Datasets/OrganCMNIST/test\\9\\image_7420.png', '../Datasets/OrganCMNIST/test\\6\\image_6127.png', '../Datasets/OrganCMNIST/test\\9\\image_4559.png', '../Datasets/OrganCMNIST/test\\1\\image_4733.png', '../Datasets/OrganCMNIST/test\\10\\image_2693.png', '../Datasets/OrganC

Image Paths:  ['../Datasets/OrganCMNIST/test\\7\\image_221.png', '../Datasets/OrganCMNIST/test\\2\\image_633.png', '../Datasets/OrganCMNIST/test\\7\\image_4758.png', '../Datasets/OrganCMNIST/test\\4\\image_3788.png', '../Datasets/OrganCMNIST/test\\2\\image_4980.png', '../Datasets/OrganCMNIST/test\\9\\image_2534.png', '../Datasets/OrganCMNIST/test\\6\\image_264.png', '../Datasets/OrganCMNIST/test\\4\\image_5065.png', '../Datasets/OrganCMNIST/test\\8\\image_5936.png', '../Datasets/OrganCMNIST/test\\7\\image_5802.png', '../Datasets/OrganCMNIST/test\\5\\image_564.png', '../Datasets/OrganCMNIST/test\\8\\image_3530.png', '../Datasets/OrganCMNIST/test\\7\\image_7186.png', '../Datasets/OrganCMNIST/test\\1\\image_1315.png', '../Datasets/OrganCMNIST/test\\9\\image_7420.png', '../Datasets/OrganCMNIST/test\\6\\image_6127.png', '../Datasets/OrganCMNIST/test\\9\\image_4559.png', '../Datasets/OrganCMNIST/test\\1\\image_4733.png', '../Datasets/OrganCMNIST/test\\10\\image_2693.png', '../Datasets/OrganC

In [6]:
print(result_list[0].image_textual_description)

Technical Quality: {'image_modality': 'CT (Computed Tomography)', 'positioning': 'Supine position with axial orientation', 'image_quality': 'Adequate with minimal artifacts'}
Anatomical Description: {'location': 'Abdominal cavity, specifically the region of the liver and surrounding structures', 'anatomical_landmarks': ['Liver', 'Diaphragm', 'Kidneys'], 'orientation': 'Axial plane'}
Findings: {'composition': {'density_characteristics': 'Hypodense areas suggestive of fluid or low-density lesions', 'internal_architecture': 'Complex internal architecture with varying densities'}, 'morphology': {'shape': 'Irregular shapes noted in the hypodense regions', 'margins': 'Well-defined margins in some areas, indistinct in others', 'size': 'Variable sizes of lesions, largest measuring approximately 3 cm'}, 'distribution': {'spatial_arrangement': 'Lesions distributed throughout the liver with some adjacent to the diaphragm', 'relationship_to_landmarks': 'Lesions are located inferior to the diaphrag

In [7]:
x_sam = pd.DataFrame(exp_results)

In [8]:
pd.DataFrame(exp_results).to_csv(f"{DATASET_NAME}_results_Algorithm5_EXP1_batch_size_1.csv", index=False)